# yelp capstone project
Consist of 5 tables e.g Business, Review, User, Tip, Checkin.\
For dataset details kindly [visit](https://www.yelp.com/dataset/documentation/main)\
Bootcamper : Robert, Sining, Theresa, Kamruddin

The project was initialized and capstoned to pass the knowledge and learning into application.

Now we are importing necessary packages 

In [1]:
import json
import pandas as pd

Opening json file from business dataset together with unpacking 'arrributes' columns to find to particular hypothetic columns.

In [2]:
data = []

with open('data/yelp_academic_dataset_business.json') as f:

    for line in f:
        
        #Parse the JSON data
        json_dict = json.loads(line)

        if json_dict['attributes']:
            if 'BusinessParking' in json_dict['attributes'].keys():

                json_dict['attributes']['BusinessParking'] = eval(json_dict['attributes']['BusinessParking'])

        data.append(json_dict)

normlize dataset

In [3]:
biz_df = pd.json_normalize(data)

In [4]:
biz_df.shape
# now we get 150,346 rows and 65 columns in total.

(150346, 65)

## Filtering out dataset by state PA (Execute Next 3 steps )

In [5]:
#1 create function for filtering by state

def state_filter(dataframe, column, state):
    businesses_new = dataframe[dataframe[column] == state].copy()
    return businesses_new

In [6]:
#2 filter for all businesses in Pennsylvania 

biz_df = state_filter(biz_df, 'state', 'PA')

In [7]:
#3 delete the columns 'address', 'city', 'state', 'postal_code', 'is_open'

biz_df.drop(columns=['address', 'city', 'state', 'postal_code', 'is_open'], inplace=True, errors='ignore')

Then the file will narrow down to 34,039 rows from 150,346 rows.

In [8]:
biz_df.shape

(34039, 60)

## Dropping empty category rows and reset index on existing dataframe

In [9]:
# drop row that have no category labels
biz_df.dropna(subset='categories', inplace=True)
# reset index
biz_df.reset_index(drop=True, inplace=True)

In [10]:
# import category labels from CSV and filter dataframe to keep row that have category label Approx 2 min
cat_data = pd.read_csv('data/categories.csv', header=None)
categories = cat_data[0].tolist()
filtered_df = pd.DataFrame()

for index, item in biz_df['categories'].items():

    for category in categories:
        if category in item:
            filtered_df = pd.concat([filtered_df, biz_df[index:index+1]])
            break

In [11]:
filtered_df.columns

Index(['business_id', 'name', 'latitude', 'longitude', 'stars', 'review_count',
       'categories', 'hours', 'attributes.ByAppointmentOnly',
       'attributes.BusinessAcceptsCreditCards', 'hours.Monday',
       'hours.Tuesday', 'hours.Wednesday', 'hours.Thursday', 'hours.Friday',
       'hours.Saturday', 'attributes.BikeParking',
       'attributes.RestaurantsPriceRange2', 'attributes.CoatCheck',
       'attributes.RestaurantsTakeOut', 'attributes.RestaurantsDelivery',
       'attributes.Caters', 'attributes.WiFi',
       'attributes.BusinessParking.garage',
       'attributes.BusinessParking.street',
       'attributes.BusinessParking.validated',
       'attributes.BusinessParking.lot', 'attributes.BusinessParking.valet',
       'attributes.WheelchairAccessible', 'attributes.HappyHour',
       'attributes.OutdoorSeating', 'attributes.HasTV',
       'attributes.RestaurantsReservations', 'attributes.DogsAllowed',
       'hours.Sunday', 'attributes.Alcohol', 'attributes.GoodForKids',
 

### Cleaning columns from business table

In [12]:
#1 dropping less important columns
filtered_df.drop(['hours','attributes.ByAppointmentOnly', 'attributes.BikeParking','attributes.CoatCheck',
       'attributes.Caters', 'attributes.WiFi', 'attributes.WheelchairAccessible', 'attributes.HappyHour', 'attributes.HasTV', 'attributes.DogsAllowed', 'attributes.GoodForKids',
       'attributes.RestaurantsAttire','attributes.NoiseLevel', 'attributes.GoodForMeal', 'attributes.BusinessAcceptsBitcoin', 'attributes.Smoking','attributes.Music',
       'attributes.GoodForDancing', 'attributes.AcceptsInsurance', 'attributes.BestNights', 'attributes.BYOB', 'attributes.Corkage', 'attributes.BYOBCorkage',
       'attributes.HairSpecializesIn','attributes.AgesAllowed','attributes.DietaryRestrictions','attributes.BusinessParking','attributes', 'attributes.Ambience'], axis=1, inplace=True)

In [13]:
filtered_df.columns

Index(['business_id', 'name', 'latitude', 'longitude', 'stars', 'review_count',
       'categories', 'attributes.BusinessAcceptsCreditCards', 'hours.Monday',
       'hours.Tuesday', 'hours.Wednesday', 'hours.Thursday', 'hours.Friday',
       'hours.Saturday', 'attributes.RestaurantsPriceRange2',
       'attributes.RestaurantsTakeOut', 'attributes.RestaurantsDelivery',
       'attributes.BusinessParking.garage',
       'attributes.BusinessParking.street',
       'attributes.BusinessParking.validated',
       'attributes.BusinessParking.lot', 'attributes.BusinessParking.valet',
       'attributes.OutdoorSeating', 'attributes.RestaurantsReservations',
       'hours.Sunday', 'attributes.Alcohol',
       'attributes.RestaurantsTableService',
       'attributes.RestaurantsGoodForGroups', 'attributes.DriveThru',
       'attributes.Open24Hours', 'attributes.RestaurantsCounterService'],
      dtype='object')

In [14]:
#2 renaming the header
filtered_df.rename(columns={'attributes.BusinessAcceptsCreditCards': 'accepts_credit_cards','hours.Monday':'monday', 'hours.Tuesday':'tuesday','hours.Wednesday':'wednesday',
                       'hours.Thursday':'thursday','hours.Friday':'friday','hours.Saturday':'saturday','hours.Sunday':'sunday','attributes.RestaurantsPriceRange2':'price_range',
                       'attributes.RestaurantsTakeOut':'take_out','attributes.RestaurantsDelivery':'delivery','attributes.BusinessParking.garage':'parking_garage',
                       'attributes.BusinessParking.street':'parking_street','attributes.BusinessParking.validated':'parking_validated','attributes.BusinessParking.lot':'parking_lot',
                       'attributes.BusinessParking.valet':'parking_valet','attributes.OutdoorSeating':'outdoor_seating','attributes.RestaurantsReservations':'seats_reservations',
                       'attributes.Alcohol':'alcohol','attributes.Ambience':'ambience','attributes.RestaurantsTableService':'table_service','attributes.RestaurantsGoodForGroups':'for_groups',
                       'attributes.DriveThru':'drive_thru','attributes.Open24Hours':'open_24_hours','attributes.RestaurantsCounterService':'counter_service'}, inplace=True)

In [15]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15830 entries, 0 to 34012
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   business_id           15830 non-null  object 
 1   name                  15830 non-null  object 
 2   latitude              15830 non-null  float64
 3   longitude             15830 non-null  float64
 4   stars                 15830 non-null  float64
 5   review_count          15830 non-null  int64  
 6   categories            15830 non-null  object 
 7   accepts_credit_cards  14370 non-null  object 
 8   monday                11136 non-null  object 
 9   tuesday               12051 non-null  object 
 10  wednesday             12766 non-null  object 
 11  thursday              13017 non-null  object 
 12  friday                13103 non-null  object 
 13  saturday              12828 non-null  object 
 14  price_range           13419 non-null  object 
 15  take_out              13

In [16]:
filtered_df.columns

Index(['business_id', 'name', 'latitude', 'longitude', 'stars', 'review_count',
       'categories', 'accepts_credit_cards', 'monday', 'tuesday', 'wednesday',
       'thursday', 'friday', 'saturday', 'price_range', 'take_out', 'delivery',
       'parking_garage', 'parking_street', 'parking_validated', 'parking_lot',
       'parking_valet', 'outdoor_seating', 'seats_reservations', 'sunday',
       'alcohol', 'table_service', 'for_groups', 'drive_thru', 'open_24_hours',
       'counter_service'],
      dtype='object')

### Cleaning data inside the cells

In [17]:
filtered_df['alcohol'].unique()

array(["u'none'", nan, "'full_bar'", "u'full_bar'", "'none'",
       "'beer_and_wine'", "u'beer_and_wine'", 'None'], dtype=object)

In [18]:
filtered_df['alcohol'] = filtered_df['alcohol'].str.replace("u'", "").str.replace("'", "")

In [19]:
filtered_df['alcohol'].unique()

array(['none', nan, 'full_bar', 'beer_and_wine', 'None'], dtype=object)

### copying the dataframe to not loose the original df and find parking availability 

In [20]:
filter_df = filtered_df.copy()

In [21]:
# creating function to check the Values either true in any case
def parking_check(row):
    if row['parking_garage'] == True or row['parking_street'] == True or row['parking_validated'] == True or row['parking_lot'] == True or row['parking_valet'] == True:
        return True
    else:
        return False

In [22]:
# running parking check by row if anywhere found True returns True in a new Column else all returning False.
filter_df['parking_available'] = filter_df.apply(parking_check, axis=1)

In [23]:
# dropping all the parking extensions from the table to be in more cleaned way
filter_df.drop(['parking_garage', 'parking_street','parking_validated','parking_lot','parking_valet'], axis=1, inplace=True)

In [24]:
filter_df

,business_id,name,latitude,longitude,stars,review_count,categories,accepts_credit_cards,monday,tuesday,...,outdoor_seating,seats_reservations,sunday,alcohol,table_service,for_groups,drive_thru,open_24_hours,counter_service,parking_available
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,39.955505,-75.155564,4.0,80,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",False,7:0-20:0,7:0-20:0,...,False,NaN,7:0-21:0,none,NaN,NaN,NaN,NaN,NaN,True
1,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,40.338183,-75.471659,4.5,13,"Brewpubs, Breweries, Food",True,NaN,NaN,...,NaN,NaN,12:0-18:0,NaN,NaN,NaN,NaN,NaN,NaN,True
2,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,39.953949,-75.143226,4.0,245,"Sushi Bars, Restaurants, Japanese",True,NaN,13:30-22:0,...,True,True,13:30-22:0,full_bar,True,True,NaN,NaN,NaN,True
3,ROeacJQwBeh05Rqg7F6TCg,BAP,39.943223,-75.162568,4.5,205,"Korean, Restaurants",True,11:30-20:30,11:30-20:30,...,None,False,NaN,none,True,False,NaN,NaN,NaN,True
5,QdN72BWoyFypdGJhhI5r7g,Bar One,39.939825,-75.157447,4.0,65,"Cocktail Bars, Bars, Italian, Nightlife, Resta...",True,16:0-0:0,16:0-0:0,...,False,False,11:0-0:0,full_bar,True,True,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34002,gPr1io7ks0Eo3FDsnDTYfg,Tata Cafe,40.060414,-75.191084,4.0,21,"Sandwiches, Restaurants, Italian",True,7:0-17:0,7:0-17:0,...,True,False,8:0-17:0,none,False,True,NaN,NaN,NaN,False
34003,wVxXRFf10zTTAs11nr4xeA,PrimoHoagies,40.032483,-75.214430,3.0,55,"Restaurants, Specialty Food, Food, Sandwiches,...",True,10:0-21:0,10:0-21:0,...,False,False,10:0-19:0,none,NaN,False,NaN,NaN,NaN,True
34006,8n93L-ilMAsvwUatarykSg,Kitchen Gia,39.951018,-75.198240,3.0,22,"Coffee & Tea, Food, Sandwiches, American (Trad...",True,9:0-19:30,9:0-19:30,...,True,False,11:0-19:30,none,False,True,NaN,NaN,NaN,False
34008,l9eLGG9ZKpLJzboZq-9LRQ,Wawa,39.925656,-75.310344,3.0,11,"Restaurants, Sandwiches, Convenience Stores, C...",True,0:0-0:0,0:0-0:0,...,NaN,NaN,0:0-0:0,NaN,NaN,NaN,NaN,NaN,NaN,True


### Now we have 15,830 rows and 28 analysable columns for further process.
### Calling review date and filter selected business_id

In [69]:
filter_df['business_id'].nunique()

15830

In [66]:
import pandas as pd

filtered = filter_df['business_id'].to_list()
column_names = ['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date']
chunks = pd.read_json('data/y_review.json', lines=True, chunksize=1500)

with open('data/review_filter.csv', 'w') as f:
    header = ','.join(column_names)
    f.write(header + '\n')

    for chunk in chunks:
        df = pd.DataFrame(chunk)
        # Append all the rows in the chunk to the CSV file, if the business ID is in the filtered list.
        df[df['business_id'].isin(filtered)].to_csv(f, mode='a', header=False, index=False)


In [68]:
output = pd.read_csv('data/review_filter.csv')
output.shape

(1205562, 9)

In [70]:
output['business_id'].nunique()

15830

In [71]:
output['user_id'].nunique()

327497

['user_id','name','review_count','yelping_since','friends','useful','funny','cool','fans','elite','average_stars','compliment_hot','compliment_more','compliment_profile','compliment_cute','compliment_list','compliment_note','compliment_plain','compliment_cool','compliment_funny','compliment_writer','compliment_photos']

In [72]:
filtered_user = output['user_id'].to_list()
column_user = ['user_id', 'name', 'review_count', 'yelping_since', 'friends', 'useful', 'funny', 'cool', 'fans', 'elite', 'average_stars', 'compliment_hot', 'compliment_more', 'compliment_profile', 'compliment_cute', 'compliment_list', 'compliment_note', 'compliment_plain', 'compliment_cool', 'compliment_funny', 'compliment_writer', 'compliment_photos']
chunks = pd.read_json('data/y_user.json', lines=True, chunksize=2000)

with open('data/user_filter.csv', 'w') as f:
    header = ','.join(column_user)
    f.write(header + '\n')

    for chunk in chunks:
        df = pd.DataFrame(chunk)
        # Append all the rows in the chunk to the CSV file, if the business ID is in the filtered list.
        df[df['user_id'].isin(filtered_user)].to_csv(f, mode='a', header=False, index=False)

In [73]:
user_df = pd.read_csv('data/user_filter.csv')
user_df.shape

(327496, 22)

In [74]:
user_df['user_id'].nunique()

327496